<a href="https://colab.research.google.com/github/stryo2/imagedenoising/blob/main/imgDenoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

# Path to the ZIP file in your Google Drive
zip_file_path = '/content/drive/My Drive/Train.zip'

# Directory where the ZIP file will be extracted
extract_dir = '/content/extracted_dataset'

# Create the directory if it does not exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("extraction done!!!")


extraction done!!!


In [ ]:
# List the contents of the extracted directory
extracted_files = os.listdir(extract_dir)
print("Extracted files and folders:", extracted_files)

# List the contents of the 'Train' directory
train_dir = os.path.join(extract_dir, 'Train')
extracted_train_files = os.listdir(train_dir)
print("Extracted files and folders in Train:", extracted_train_files)


Extracted files and folders: ['Train']
Extracted files and folders in Train: ['low', 'high']


In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Reshape, Add, Activation, Multiply, concatenate
from tensorflow.keras.models import Model


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

def create_patches(image, patch_size):
    patches = []
    h, w, _ = image.shape
    for i in range(0, h, patch_size):
        for j in range(0, w, patch_size):
            patch = image[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    return np.array(patches)

def load_and_patch_images(folder, image_size, patch_size):
    patches = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = load_img(img_path, target_size=image_size)
            img_array = img_to_array(img)
            img_patches = create_patches(img_array, patch_size)
            patches.extend(img_patches)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
    return np.array(patches)

# Parameters
image_size = (512, 512)  # Resize images to this size
patch_size = 256  # Patch size

# Define paths to image directories
high_images_dir = os.path.join(train_dir, 'high')
low_images_dir = os.path.join(train_dir, 'low')

# Check if directories exist
if not os.path.exists(high_images_dir):
    print(f"Directory not found: {high_images_dir}")
if not os.path.exists(low_images_dir):
    print(f"Directory not found: {low_images_dir}")

# Load and patch images if directories exist
if os.path.exists(high_images_dir) and os.path.exists(low_images_dir):
    X_high = load_and_patch_images(high_images_dir, image_size, patch_size)
    X_low = load_and_patch_images(low_images_dir, image_size, patch_size)

    # Normalize images
    X_high = X_high / 255.0
    X_low = X_low / 255.0

    # Split into train and test sets (you can adjust the split ratio)
    split_ratio = 0.8
    split_index = int(len(X_high) * split_ratio)

    X_train_patches = X_low[:split_index]
    y_train_patches = X_high[:split_index]
    X_test_patches = X_low[split_index:]
    y_test_patches = X_high[split_index:]

    print(f"Train data shape: {X_train_patches.shape}, {y_train_patches.shape}")
    print(f"Test data shape: {X_test_patches.shape}, {y_test_patches.shape}")
else:
    print("One or both directories do not exist. Please check the paths.")


Error loading image /content/extracted_dataset/Train/high/.DS_Store: cannot identify image file <_io.BytesIO object at 0x7e0ad03aaf70>
Error loading image /content/extracted_dataset/Train/low/.DS_Store: cannot identify image file <_io.BytesIO object at 0x7e0ad03ab380>
Train data shape: (1552, 256, 256, 3), (1552, 256, 256, 3)
Test data shape: (388, 256, 256, 3), (388, 256, 256, 3)


In [ ]:
class Dataloder(tf.keras.utils.Sequence):
    def __init__(self, X, y, batch_size=1, shuffle=False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(X))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, i):
        indexes = self.indexes[i * self.batch_size : (i+1) * self.batch_size]
        batch_x = self.X[indexes]
        batch_y = self.y[indexes]
        return batch_x, batch_y

    def __len__(self):
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

# Create data loaders
batch_size = 8
train_dataloader = Dataloder(X_train_patches, y_train_patches, batch_size, shuffle=True)
test_dataloader = Dataloder(X_test_patches, y_test_patches, batch_size, shuffle=False)


In [ ]:
class EAM(tf.keras.layers.Layer):
  def __init__(self,**kwargs):
    super().__init__(**kwargs)

    self.conv1 = Conv2D(64, (3,3), dilation_rate=1,padding='same',activation='relu')
    self.conv2 = Conv2D(64, (3,3), dilation_rate=2,padding='same',activation='relu')

    self.conv3 = Conv2D(64, (3,3), dilation_rate=3,padding='same',activation='relu')
    self.conv4 = Conv2D(64, (3,3), dilation_rate=4,padding='same',activation='relu')

    self.conv5 = Conv2D(64, (3,3),padding='same',activation='relu')

    self.conv6 = Conv2D(64, (3,3),padding='same',activation='relu')
    self.conv7 = Conv2D(64, (3,3),padding='same')

    self.conv8 = Conv2D(64, (3,3),padding='same',activation='relu')
    self.conv9 = Conv2D(64, (3,3),padding='same',activation='relu')
    self.conv10 = Conv2D(64, (1,1),padding='same')

    self.gap = GlobalAveragePooling2D()

    self.conv11 = Conv2D(64, (3,3),padding='same',activation='relu')
    self.conv12 = Conv2D(64, (3,3),padding='same',activation='sigmoid')

  def call(self,input):
    conv1 = self.conv1(input)
    conv1 = self.conv2(conv1)

    conv2 = self.conv3(input)
    conv2 = self.conv4(conv2)

    concat = concatenate([conv1,conv2])
    conv3 = self.conv5(concat)
    add1 = Add()([input,conv3])

    conv4 = self.conv6(add1)
    conv4 = self.conv7(conv4)
    add2 = Add()([conv4,add1])
    add2 = Activation('relu')(add2)

    conv5 = self.conv8(add2)
    conv5 = self.conv9(conv5)
    conv5 = self.conv10(conv5)
    add3 = Add()([add2,conv5])
    add3 = Activation('relu')(add3)

    gap = self.gap(add3)
    gap = Reshape((1,1,64))(gap)
    conv6 = self.conv11(gap)
    conv6 = self.conv12(conv6)

    mul = Multiply()([conv6, add3])
    out = Add()([input,mul])
    return out

def psnr(y_true, y_pred):
    max_pixel = 1.0
    return tf.image.psnr(y_true, y_pred, max_val=max_pixel)

tf.keras.backend.clear_session()
input = Input(shape=(256, 256, 3))

conv1 = Conv2D(64, (3,3),padding='same')(input)
eam1 = EAM()(conv1)
eam2 = EAM()(eam1)
eam3 = EAM()(eam2)
eam4 = EAM()(eam3)
conv2 = Conv2D(3, (3,3),padding='same')(eam4)
out = Add()([conv2,input])

RIDNet = Model(input,out)
RIDNet.compile(optimizer=tf.keras.optimizers.Adam(1e-03), loss=tf.keras.losses.MeanSquaredError(),metrics=[psnr])

In [ ]:
# Train the model
RIDNet.fit(train_dataloader, validation_data=test_dataloader, epochs=15)

Epoch 1/15
194/194 [==============================] - 344s 2s/step - loss: 0.0377 - psnr: 16.5303 - val_loss: 0.0278 - val_psnr: 17.3093
Epoch 2/15
194/194 [==============================] - 334s 2s/step - loss: 0.0258 - psnr: 17.8718 - val_loss: 0.0222 - val_psnr: 18.2577
Epoch 3/15
194/194 [==============================] - 316s 2s/step - loss: 0.0197 - psnr: 19.1108 - val_loss: 0.0188 - val_psnr: 19.0022
Epoch 4/15
194/194 [==============================] - 315s 2s/step - loss: 0.0192 - psnr: 19.2879 - val_loss: 0.0178 - val_psnr: 19.4708
Epoch 5/15
194/194 [==============================] - 315s 2s/step - loss: 0.0192 - psnr: 19.2435 - val_loss: 0.0177 - val_psnr: 19.7338
Epoch 6/15
194/194 [==============================] - 332s 2s/step - loss: 0.0181 - psnr: 19.6381 - val_loss: 0.0172 - val_psnr: 19.7545
Epoch 7/15
194/194 [==============================] - 315s 2s/step - loss: 0.0175 - psnr: 19.6790 - val_loss: 0.0167 - val_psnr: 20.0072
Epoch 8/15
194/194 [=====================

In [ ]:
results = RIDNet.evaluate(test_dataloader)
print(f"Test Loss: {results[0]}")
print(f"Test PSNR: {results[1]}")

97/97 [==============================] - 24s 247ms/step - loss: 0.0161 - psnr: 19.9635
Test Loss: 0.016132060438394547
Test PSNR: 19.963502883911133
